# Fitts Law Analysis

In [25]:
import json
import pandas as pd
import numpy as np

f = open('./input/interaction_data.json', 'r')
data = json.load(f)
df = pd.json_normalize(data, record_path=['logs'])

print(df.shape)
df.head()

(9, 7)


,clickNum,clickTime,timeDiff,buttonHeight,buttonWidth,buttonX,buttonY
0,1,1739983870934,0,50,50,30,60
1,2,1739983871612,678,297,82,101,229
2,3,1739983872400,788,289,19,183,128
3,4,1739983873039,639,99,63,266,319
4,5,1739983873630,591,188,295,252,87


## Fitts's law
$MT = a + b * log2(2D/W)$

MT is the average time to complete the movement.

- a and b are constants that depend on the choice of input device and are usually determined empirically by regression analysis. 
- a defines the intersection on the y axis and is often interpreted as a delay. 
- The b parameter is a slope and describes an acceleration. 
- Both parameters show the linear dependency in Fitts's law.[10]

- ID is the index of difficulty.
- D is the distance from the starting point to the center of the target.
- W is the width of the target measured along the axis of motion. 
- W can also be thought of as the allowed error tolerance in the final position, since the final point of the motion must fall within ±W⁄2 of the target's center.

Imagine a mouse cursor on a screen needing to click a button 100 pixels away (D=100) with a width of 50 pixels (W=50). 

W = buttonWidth 


### Distance Formula

Regular Formula:

$d = \sqrt {\left( {x_1 - x_2 } \right)^2 + \left( {y_1 - y_2 } \right)^2 }$


Project Formula:

$D = \sqrt {\left( {buttonX_1 - buttonX_2 } \right)^2 + \left( {buttonY_1 - buttonY_2 } \right)^2 }$

Where the first sample is the initial starting point, then X1 is the previous button location

In [ ]:
preprocessing = df.copy()
preprocessing['w'] = preprocessing['buttonWidth']

# fields for distance calculation
preprocessing['x1'] = preprocessing['buttonX'].shift(1)
preprocessing['y1'] = preprocessing['buttonY'].shift(1)
preprocessing['(x1-x2)^2'] = preprocessing.apply(
                    lambda row: (row['x1'] - row['buttonX'])**2 if pd.notna(row['x1']) else row['buttonX'], axis=1
                )
preprocessing['(y1-y2)^2'] = preprocessing.apply(
                    lambda row: (row['y1'] - row['buttonY'])**2 if pd.notna(row['y1']) else row['buttonY'], axis=1
                )
preprocessing['d'] = np.sqrt(preprocessing['(x1-x2)^2'] + preprocessing['(y1-y2)^2'])

preprocessing.head()

,clickNum,clickTime,timeDiff,buttonHeight,buttonWidth,buttonX,buttonY,w,x1,y1,(x1-x2)^2,(y1-y2)^2,d
0,1,1739983870934,0,50,50,30,60,50,NaN,NaN,30.0,60.0,9.486833
1,2,1739983871612,678,297,82,101,229,82,30.0,60.0,5041.0,28561.0,183.308483
2,3,1739983872400,788,289,19,183,128,19,101.0,229.0,6724.0,10201.0,130.096118
3,4,1739983873039,639,99,63,266,319,63,183.0,128.0,6889.0,36481.0,208.254652
4,5,1739983873630,591,188,295,252,87,295,266.0,319.0,196.0,53824.0,232.422030


In [ ]:
final_df = preprocessing.copy()
final_df = final_df[['timeDiff', 'w', 'd']]
final_df = final_df[1:] # excluding first record

print(final_df.shape)
final_df.head()

(8, 4)


,timeDiff,buttonHeight,w,d
1,678,297,82,183.308483
2,788,289,19,130.096118
3,639,99,63,208.254652
4,591,188,295,232.422030
5,561,217,240,236.907155


In [ ]:
# creating groups

group_conditions = [
    (final_df['buttonHeight'])
]
